In [1]:
# Run the code below each time the notebook is started or restarted to ensure
# that if you change any code in the library, this notebook will use the latest
# version of the library code.
%load_ext autoreload
%autoreload 2

In [2]:
# contains scraping and cleaning functions
from helpers import *
# setting display settings so that the maximum amount of data will show
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [3]:
# scraping population table
population = table_scrape("https://en.wikipedia.org/wiki/List_of_countries_by_past_and_projected_future_population#Estimates_between_the_years_1985_and_2015_(in_thousands)", 1)
# dropping unnecessary years
population.drop(["1985", "1990", "1995", "2000", "%", "%.1", "%.2", "%.3", "%.4","%.5", "%.6"], axis = 1, inplace = True)
# renaming columns to be associate with the correct olympic years
population.rename(columns = {"2005" : "pop-2004", "2010" : "pop-2008", "2015" : "pop-2016"}, inplace = True)
# creating a column for the population in 2012 and duplicating 2008 into it
population['pop-2012'] = population['pop-2008']
# reordering population dataframe to be chronological
population = population[['Country (or dependent territory)', 'pop-2004', 'pop-2008', 'pop-2012', 'pop-2016']]

In [4]:
# scraping GDP tables (IMF statistics)
gdp_2000 = table_scrape("https://en.wikipedia.org/wiki/List_of_countries_by_past_and_projected_GDP_(PPP)_per_capita", 2)
gdp_2010 = table_scrape("https://en.wikipedia.org/wiki/List_of_countries_by_past_and_projected_GDP_(PPP)_per_capita", 3)

In [5]:
# dropping unnecssary years
gdp_2000.drop(["2000", "2001", "2002", "2003", "2005", "2006", "2007", "2009"], axis = 1, inplace = True)
gdp_2010.drop(["2010", "2011", "2013", "2014", "2015", "2017", "2018", "2019"], axis = 1, inplace = True)

In [6]:
# merging both GDP tables and renaming to include GDP in all column titles
gdp_total = gdp_2000.merge(gdp_2010,how="left",left_on="Country (or dependent territory)",right_on="Country (or dependent territory)")
gdp_total.rename(columns = {"Country (or dependent territory)":"Country", "2004" : "GDP-2004", "2008" : "GDP-2008", "2012" : "GDP-2012", "2016" : "GDP-2016"}, inplace = True)

# Rename the UK and Taiwan to the names they compete in the olympics under
gdp_total.replace({"United Kingdom" : "Great Britain"}, inplace = True)
gdp_total.replace({"Taiwan" : "Chinese Taipei"}, inplace = True)

# Use the UN's GDP per capita data for Cuba and North Korea
# Source: https://en.wikipedia.org/wiki/List_of_countries_by_past_and_projected_GDP_(nominal)_per_capita#UN_estimates_between_2000_and_2009
cuba_row = {"Country":"Cuba", "GDP-2004":3399, "GDP-2008":5386, "GDP-2012":6448, "GDP-2016":7657}
gdp_total = gdp_total.append(cuba_row, ignore_index=True)
north_korea_row = {"Country":"North Korea", "GDP-2004":473, "GDP-2008":551, "GDP-2012":643, "GDP-2016":642}
gdp_total = gdp_total.append(north_korea_row, ignore_index=True)

In [7]:
# Average the GDP per capita for Serbia and Montenegro, because they compete
# in the olympics together as "Serbia and Montenegro"
serbia_index = gdp_total.index[gdp_total['Country'] == "Serbia"].item()
montenegro_index = gdp_total.index[gdp_total['Country'] == "Montenegro"].item()
ave_2004 = (gdp_total.loc[serbia_index,"GDP-2004"].item()+gdp_total.loc[montenegro_index,"GDP-2004"].item())/2
ave_2008 = (gdp_total.loc[serbia_index,"GDP-2008"].item()+gdp_total.loc[montenegro_index,"GDP-2008"].item())/2
ave_2012 = (gdp_total.loc[serbia_index,"GDP-2012"].item()+gdp_total.loc[montenegro_index,"GDP-2012"].item())/2
ave_2016 = (gdp_total.loc[serbia_index,"GDP-2016"].item()+gdp_total.loc[montenegro_index,"GDP-2016"].item())/2
s_m_row = {"Country":"Serbia and Montenegro", "GDP-2004":ave_2004, "GDP-2008":ave_2008, "GDP-2012":ave_2012, "GDP-2016":ave_2016}
gdp_total = gdp_total.append(s_m_row, ignore_index=True)

In [7]:
# print(gdp_total)

In [8]:
# Scraping medals tables for summer Olympics from 2004-2016 and renaming host countries to no longer include asterisk
medal_2016 = medal_clean(table_scrape("https://en.wikipedia.org/wiki/2016_Summer_Olympics_medal_table"), "2016")
medal_2016.replace({"Brazil*" : "Brazil"}, inplace = True)
medal_2012 = medal_clean(table_scrape("https://en.m.wikipedia.org/wiki/2012_Summer_Olympics_medal_table"), "2012")
medal_2012.replace({"Great Britain*" : "Great Britain"}, inplace = True)
medal_2008 = medal_clean(table_scrape("https://en.m.wikipedia.org/wiki/2008_Summer_Olympics_medal_table"), "2008")
medal_2008.replace({"China*" : "China"}, inplace = True)
medal_2004 = medal_clean(table_scrape("https://en.m.wikipedia.org/wiki/2004_Summer_Olympics_medal_table"), "2004")
medal_2004.rename(columns = {"Nation" : "NOC"}, inplace = True)
medal_2004.replace({"Greece*" : "Greece"}, inplace = True)

In [14]:
# merging all medals tables to create a total medals data frame
medal_total = medal_2012.merge(medal_2008,how="outer",left_on="NOC",right_on="NOC")
medal_total = medal_total.merge(medal_2016,how="outer",left_on="NOC",right_on="NOC")
medal_total = medal_total.merge(medal_2004,how="outer",left_on="NOC",right_on="NOC")

In [16]:
# mergining medals, GDP, and pop. All based on medals as not all countries compete in the Olympics
total = medal_total.merge(gdp_total,how="left",left_on="NOC",right_on="Country")
total = total.merge(population,how="left",left_on="NOC",right_on="Country (or dependent territory)")

In [17]:
print(total)

                              NOC  Gold-2012  Silver-2012  Bronze-2012  \
0                   United States       47.0         27.0         30.0   
1                           China       39.0         31.0         22.0   
2                   Great Britain       29.0         18.0         18.0   
3                          Russia       19.0         21.0         27.0   
4                     South Korea       13.0          9.0          8.0   
5                         Germany       11.0         20.0         13.0   
6                          France       11.0         11.0         13.0   
7                       Australia        8.0         15.0         12.0   
8                           Italy        8.0          9.0         11.0   
9                         Hungary        8.0          4.0          6.0   
10                          Japan        7.0         14.0         17.0   
11                           Iran        7.0          5.0          1.0   
12                    Netherlands     

In [12]:
total.to_csv("olympic_and_GDP_data.csv", index=False)

In [18]:
# test_data = pd.read_csv("olympic_and_GDP_data.csv")
# print(test_data)